### 3.1. Linear regression

#### 3.1.1. Basics

$\hat y = w_1 x_1 + \ldots + w_d x_d + b$

$\hat y = \textbf w^T \textbf x + b$

w is (2,1), x is (2,1) (only one observation)

It is often better to represent in matrix/vector form. 

$\hat {\textbf y} = \textbf {X}^T\textbf{w} + b$

$\hat {\textbf y} \in \mathbb{R}^{n\times 1}, \mathbf X \in \mathbb{R}^{n\times d}, \textbf w \in \mathbb{R}^{d\times 1}, b \in \mathbb{R}^{1\times 1}$, and $n$ is the number of observations, while $d$ is the number of features

#### 3.1.1.2. Loss function

$l^{(i)}(\textbf w, b) = \frac{1}{2}\left( \hat y ^{(i)} - y^{(i)}  \right)^2$

$L(\textbf w, b) = \frac{1}{n} \sum_{i=1}^n l^{(i)}(\textbf w, b) = \frac{1}{n} \sum_{i=1}^n  \frac{1}{2}\left( \textbf w^T \textbf x^{(i)} + b - y^{(i)}  \right)^2$

When training the model, we seek:

$\textbf w^*, b^* = \argmin_{\textbf w, b} L(\textbf w, b)$

we can solve this by taking the bias into the parameter $\textbf w$ by appending a column of ones in the design matrix $\textbf W$. Our problem then is to minimize $||\textbf~y~-~\textbf{Xw}||^2$... this leads to solving: 

$$\textbf w^* = (\textbf{X}^T\textbf{X})^{-1} \textbf{X}^T \textbf{y}$$